# Introduction of using the `tau` package

The `tau` package provides different classes and methods to apply the theory of dialectical structures (as introduced in [Betz (2010)](https://doi.org/10.5771/9783465136293) and [Betz (2013)](https://www.springer.com/de/book/9789400745988)). 

The provided functionalities are described by two abstract classes. Implementations of `DialecticalStructure` can be used to create, manipulate and calculate properties of dialectical structures. Implementations of `Position` represent positions on a sentence pool.

The `tau` package includes different implementations of these abstract classes, which differ w.r.t. their runtime performance:

+ *DAG (directed acyclic graphs) based dialectical structures:* All important properties of the stucture are calculated and stored during instantiation. This implementation is fast for small sentence pools ($n<15$).
    + We recommend to use the following standard implementation: `DAGDialecticalStructure`.
+ *BDD (binary decision diagramm) based dialectical structures:* Important properties of the structure are calculated by using binary decision trees. This representation is comparably fast for most properties of the graph even if the sentence pool is larger ($n>10$). However, for larger sentence pools it will become more difficult to calculate all dialectically consistent positions, axiomatic bases (without a confining source) and minimal positions. (see the API-DOCs for further details (LINK NEEDED).
   + We recommend to use the following standard implementation: `BDDDialecticalStructure`.
   
   
*Source:* You can download this notebook from [here](https://github.com/debatelab/re-docs/blob/master/source/tau-docs/tutorials/tau-tutorial.ipynb).

## Instantiation of dialectical structures and positions

A dialectical structure is a tupel  $\left<S,A\right>$ of a sentence pool $S = \{ s_1, s_2, \dots, s_n, \neg s_1, \neg s_2, \dots, \neg s_n \}$ and a set $A$ of arguments.

An argument $a=(P_a, c_a)$ is defined as a pair consisting of premises $P_a \subset S$ and a conclusion $c_a \in S$.

A dialectical structure is instantiated by specifying the size $n$ of the sentence pool and a list of arguments, in which sententences are represented numerals. The minus sign is used to indicate the negation of a sentence. For instance, the list `[1,3,-4]` represents an argument with the premsises $P_a=\{s_1,s_3\}$ and the conclusion $\neg s_4$.   

In [1]:
from tau import DAGDialecticalStructure

# size of the sentencepool
n = 7
# a list of arguments
arguments = [[1, 3],[1, 4],[1, 5],[1, -6], [2, -4],[2, 5],[2, 6],[2, 7]]
# instantiation of a dialectical structure tau
tau = DAGDialecticalStructure.from_arguments(arguments, n)

A position is a binary belief state that is represented by a subset $\mathcal{A}\subset S$  over the. The internal representation depends on the implementation. However, independent of their implementation, positions can be instantiated by specifying the set of sentences the agent believes:

In [2]:
from tau import StandardPosition

# believing s_3, s_4 and s_5
belief_state_a = StandardPosition.from_set({3, 4, 5}, n)

# believing s_2 and believing that s_4 is false 
belief_state_b = StandardPosition.from_set({2,-4}, n)

The `tau` package provides several methods to manipulate dialectical structures and to determine properties of positions with regard to a given dialectical structure. For instance:

In [3]:
# implications of a believed sentences w.r.t. the 
# inferential relationships encoded in tau
print(f"Implications of {belief_state_a}: {tau.closure(belief_state_a)}")
print(f"Implications of {belief_state_b}: {tau.closure(belief_state_b)}")

# checking whether a position is consistent w.r.t. tau
print(f"Is {belief_state_a} consistent? {tau.is_consistent(belief_state_a)}")
print(f"Is {belief_state_b} consistent? {tau.is_consistent(belief_state_b)}")

# checking whether two positions are consistent with each other (w.r.t tau)
print(f"Are {belief_state_a} and {belief_state_b} consistent with each other?" +
      f" {tau.are_compatible(belief_state_a,belief_state_b)}")

Implications of {3, 4, 5}: {3, 4, 5, -2}
Implications of {2, -4}: {2, 5, 6, 7, -4, -1}
Is {3, 4, 5} consistent? True
Is {2, -4} consistent? True
Are {3, 4, 5} and {2, -4} consistent with each other? False


## Export to JSON

### Serializing `tau` objects

You can serialize `tau` positions and dialectical structures and any compounds thereof, as long as the `json` python module can handle them (e.g., lists, dictionaries).

For instance, the following code will serialize a position:

In [4]:
from tau import StandardPosition
from tau.util import tau_dumps, tau_dump
from os import getcwd, path

# serializing a position as JSON String
pos_json_str = tau_dumps(StandardPosition.from_set({1,2},4),
                         indent=4)
print(pos_json_str)

# serializing a list of positions
pos_list = [StandardPosition.from_set({1,2},4),
            StandardPosition.from_set({1,3},4),
            StandardPosition.from_set(set(),0)]
print(tau_dumps(pos_list, indent=4))

# serializing a list of position into a file
output_file_path = path.join(getcwd(),'positions.json')
with open(file=output_file_path, mode='w') as output_file:
    tau_dump(pos_list, output_file, indent=4)

{
    "n_unnegated_sentence_pool": 4,
    "position": [
        1,
        2
    ]
}
[
    {
        "n_unnegated_sentence_pool": 4,
        "position": [
            1,
            2
        ]
    },
    {
        "n_unnegated_sentence_pool": 4,
        "position": [
            1,
            3
        ]
    },
    {
        "n_unnegated_sentence_pool": 0,
        "position": []
    }
]


If important, you can save the implementation details (module and class name), which can be considered later to deserialize the objects:

In [5]:
from tau import SetBasedPosition
from tau.util import tau_dumps

# serializing a position as JSON String
pos_json_str = tau_dumps(StandardPosition.from_set({1,2},4),
                         indent=4,
                         serialize_implementation=True)
print(pos_json_str)

{
    "n_unnegated_sentence_pool": 4,
    "position": [
        1,
        2
    ],
    "module_name": "tau.bitarray_implementation",
    "class_name": "BitarrayPosition"
}


### Deserializing `tau` objects

The deserialization of objects is similarly simple. The implementation details can either be taken from the json file or can be explicitly given via parameters.

In [6]:
from tau import SetBasedPosition, BitarrayPosition
from tau.util import tau_dumps, tau_loads, tau_load
from os import getcwd, path

# serializing a position as JSON String
pos_json_str = tau_dumps(SetBasedPosition.from_set({1,2},4),
                         indent=4,
                         serialize_implementation=True)
# deserializing it
position = tau_loads(pos_json_str, use_json_specified_type = True )
print(f"{position} of type {type(position)}")

# deserializing it and using another implementation
position = tau_loads(pos_json_str,
                     position_module = 'tau',
                     position_class = 'BitarrayPosition' )
print(f"{position} of type {type(position)}")

# deserializing tau objects from a file
input_file_path = path.join(getcwd(),'positions.json')
with open(file=input_file_path, mode='r') as input_file:
    obj = tau_load(input_file)
print(obj)

{1, 2} of type <class 'tau.set_implementation.SetBasedPosition'>
{1, 2} of type <class 'tau.bitarray_implementation.BitarrayPosition'>
[{1, 2}, {1, 3}, set()]
